In [1]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.stats import rankdata
from random import seed, randint
from operator import itemgetter
import pandas as pd
import pickle
# from random import randint

## Real code

In [4]:
class Paper:
    def __init__(self,id):
        self.id = id
        self.test_cited_paper = []
        self.train_cited_paper = []

    def add_test_cited_paper(self,cited_paper_id):
        self.test_cited_paper.append(cited_paper_id)
        
    def add_train_cited_paper(self,cited_paper_id):
        self.train_cited_paper.append(cited_paper_id)
    

### for 5-fold cross validation

In [5]:
def train_eval_PaperRank(dataset_dir):
    train_dir = []
    test_dir = []

    pr5fold = {}
    rec5fold = {}
    f15fold = {}
    
    n_iter = 50
    alpha = 0.85
    fold = 1
    mrr5fold = 0
    bpref5fold = 0
    k = 20 # for precision@k, recall@k, ...@k
    
    
    for i in range(1,6):
#     for i in range(1,2):    
        train_dir.append(dataset_dir+'/train_fold'+str(i)+'.txt')
        test_dir.append(dataset_dir+'/test_fold'+str(i)+'.txt')
        
    for i in range(1,k+1):
        pr5fold[i] = 0
        rec5fold[i] = 0
        f15fold[i] = 0

    # print(train_dir)
    # print(test_dir)
    for s1,s2 in zip(train_dir,test_dir):
        print(s1,s2)
        train_data = open(s1,'r')
        test_data = open(s2,'r')
        doi2id_file = open(dataset_dir+'/id_morethan1cite.txt','r')

        lines = train_data.readlines()
        lines2 = test_data.readlines()
        doi2id = doi2id_file.readlines()
        
        total_node = len(doi2id)
        cite_graph = np.zeros((total_node,total_node))
        test_papers = {}
        ir = np.random.rand(total_node,1)
        d_vec = np.zeros((total_node,1))
        m = np.zeros((total_node,total_node))
        result = '' # store recommendation list
#         sorted_ir_dict = {}
        all_sorted_ir_dict = {}
    
        PrecAtK = {}
        RecAtK = {}
        F1AtK = {}
        mrr_allpaper = 0
        bpref_allpaper = 0
        
        for i in range(1,k+1):
            PrecAtK[i] = 0
            RecAtK[i] = 0
            F1AtK[i] = 0
            
        for line in lines2:
            edge_arr = line.split('\t')
            citing_id = int(edge_arr[0])
            cited_id = int(edge_arr[2].replace('\n',''))
#             d_vec[cited_id] = 1
        #     test_cited_paper.append(cited_id)

            if citing_id not in test_papers: 
                test_papers[citing_id] = Paper(citing_id)
            test_papers[citing_id].add_test_cited_paper(cited_id)

        for line in lines:
            edge_arr = line.split('\t')
            citing_id = int(edge_arr[0])
            cited_id = int(edge_arr[2].replace('\n',''))
            cite_graph[citing_id,cited_id] = 1
            cite_graph[cited_id,citing_id] = 1
            if citing_id in test_papers:
                test_papers[citing_id].add_train_cited_paper(cited_id)

        for a in range(0,total_node):
            col_sum = np.sum(cite_graph[:,a])
            if col_sum == 0:
                m[:,a] = 0
            else:
                m[:,a] = cite_graph[:,a]/col_sum

        for p in test_papers:
            ir = np.random.rand(total_node,1)
            d_vec = np.zeros((total_node,1))
            ir_dict = {} # store pair of paper id and score
            sorted_ir_dict = {}
            min_ir = 9999999
            ranked_paper = []
            bpref = 0
            mrr = 0
            
            for c in test_papers[p].train_cited_paper:
                d_vec[c] = 1

            for r in range(90):
                ir = alpha*np.dot(m,ir)+(1-alpha)*(d_vec/total_node)

            ir_rank = rankdata(ir)
            ir_rank = len(ir_rank)-ir_rank+1
            
            for i in range(0,len(ir)):
                ir_dict[i] = ir[i][0]

            sorted_ir = sorted(ir_dict.items(), key=itemgetter(1),reverse=True)
            
            for i in range(0,len(sorted_ir)):
#                 sorted_ir_dict[str(sorted_ir[i][0])] = str(sorted_ir[i][1])
                ranked_paper.append(sorted_ir[i][0])
#             all_sorted_ir_dict[test_papers[p].id] = sorted_ir_dict
    
#         with open('PaperRank_score/' + dataset_dir + "_fold" + str(fold) + '.pkl', 'wb') as f:
#             pickle.dump(all_sorted_ir_dict, f, pickle.HIGHEST_PROTOCOL)
            
#         fold = fold+1
#             print(sorted_ir)
#             break
            
#         break
            
#             print(sorted_ir)
#             print(ranked_paper)
            # +str(citing_id)+'\t'+ ','.join(map(str, recommended_papers))
#             result = result + str(p) + '\t' + ','.join(map(str, ranked_paper[:50])) + '\n' 
#             break
        
            for pc in test_papers[p].test_cited_paper:
                if ir_rank[pc] < min_ir:
                    min_ir = ir_rank[pc]
    
            if min_ir <= k:
                mrr = mrr + (1/min_ir)
                
            mrr_allpaper = mrr_allpaper+mrr
#             if min_ir > 10:
#                 min_ir = 0
#             else:    
#                 mrr = mrr + (1/min_ir)
                
    
            for pc in test_papers[p].test_cited_paper:
                r_higher = 0
                for rec in ranked_paper[:k]:
                    if pc == rec:
                        break
                    if pc != rec and rec not in test_papers[p].test_cited_paper:
                        r_higher = r_higher + 1
                bpref = bpref + (1-(r_higher/k))
            
            bpref = bpref/len(test_papers[p].test_cited_paper)
            bpref_allpaper = bpref_allpaper+bpref
                                 
            # loop to find precision,recall,F1 @k
            for i in range(1,k+1):
                pr = len(set(test_papers[p].test_cited_paper)&set(ranked_paper[:i]))/i
                rec = len(set(test_papers[p].test_cited_paper)&set(ranked_paper[:i]))/len(test_papers[p].test_cited_paper)
                F1 = (2*pr*rec)/(pr+rec) if pr+rec > 0 else 0
                PrecAtK[i] = PrecAtK[i] + pr
                RecAtK[i] = RecAtK[i] + rec
                F1AtK[i] = F1AtK[i] + F1
                
        mrr_allpaper = mrr_allpaper/len(test_papers) # divide mrr by total citations
        mrr5fold = mrr5fold+mrr_allpaper
        
        bpref_allpaper = bpref_allpaper/len(test_papers)
        bpref5fold = bpref5fold + bpref_allpaper
        
        for i in range(1,k+1):
            PrecAtK[i] = PrecAtK[i]/len(test_papers)
            RecAtK[i] = RecAtK[i]/len(test_papers)
            F1AtK[i] = F1AtK[i]/len(test_papers)

        for i in range(1,k+1):
            pr5fold[i] = pr5fold[i] + PrecAtK[i]
            rec5fold[i] = rec5fold[i] + RecAtK[i]
            f15fold[i] = f15fold[i] + F1AtK[i]

        print(ranked_paper)
        
    for i in range(1,k+1):
        pr5fold[i] = pr5fold[i]/5.0
        rec5fold[i] = rec5fold[i]/5.0
        f15fold[i] = f15fold[i]/5.0
        
    mrr5fold = mrr5fold/5.0
    bpref5fold = bpref5fold/5.0
#         result_file = open('Paperrank_result/'+'recommendation result_'+dataset_dir+'_fold'+str(fold)+'.txt','w')
#         result_file.write(result)
#         fold = fold+1
    # print precision
    print(pr5fold.values())
    # print recall
    print(rec5fold.values())
    # print f1
    print(f15fold.values())
    # print MRR, Bpref
    print(mrr5fold,'\n',bpref5fold)
    
# for dataset in ['algo_citation', 'topic_citation', 'CiteSeer_umd' , 'CiteULike']:
#     print(dataset) 
# dataset = 'CiteSeer_umd' # or CiteULike, topic_citation
# train_eval_PaperRank(dataset)

In [6]:
train_eval_PaperRank('CiteULike_5_fold')

CiteULike_5_fold/train_fold1.txt CiteULike_5_fold/test_fold1.txt
[3296, 1622, 1892, 787, 619, 1507, 1309, 1079, 314, 352, 2686, 2676, 891, 2532, 1465, 1564, 1109, 2858, 792, 1729, 2385, 874, 2041, 2178, 887, 2456, 3041, 726, 2110, 84, 2880, 2053, 1556, 2105, 2584, 1621, 2936, 2292, 2383, 279, 1926, 1080, 2689, 2013, 2309, 2360, 721, 2188, 933, 663, 2857, 1140, 4306, 3038, 2186, 977, 420, 696, 2536, 2897, 2193, 1262, 565, 2829, 3037, 2399, 2397, 19, 2426, 679, 1084, 3302, 3843, 346, 1691, 1399, 1702, 1966, 3742, 757, 993, 1648, 2132, 1609, 2418, 2216, 2859, 1000, 2222, 1009, 2795, 3417, 2793, 1474, 3508, 2382, 1447, 1669, 1426, 3174, 3265, 788, 2200, 3336, 2910, 1406, 2184, 250, 1558, 1225, 2911, 1941, 773, 1774, 2650, 1525, 2791, 1786, 1663, 114, 719, 3040, 4399, 1006, 1391, 692, 3257, 1054, 2531, 2633, 645, 2576, 2811, 1187, 613, 867, 2015, 3225, 3980, 1987, 2929, 2600, 2626, 2431, 1386, 1817, 676, 1629, 552, 4484, 1584, 1900, 2661, 2694, 1151, 258, 2709, 2803, 973, 83, 2796, 1963, 10

In [7]:
train_eval_PaperRank('TopicCitation_5_fold')

TopicCitation_5_fold/train_fold1.txt TopicCitation_5_fold/test_fold1.txt
[1346, 174, 182, 216, 400, 615, 1546, 429, 454, 872, 1634, 992, 291, 1026, 437, 416, 976, 1070, 1025, 569, 453, 188, 969, 1564, 415, 604, 140, 527, 161, 894, 597, 1085, 340, 701, 916, 410, 470, 700, 773, 738, 343, 397, 657, 1734, 1020, 513, 560, 252, 1078, 508, 1665, 568, 1574, 564, 923, 1590, 1537, 1599, 798, 282, 449, 432, 1610, 1596, 1565, 924, 683, 471, 1088, 629, 641, 422, 1068, 593, 599, 778, 829, 516, 237, 242, 1084, 1095, 90, 1092, 546, 465, 834, 405, 602, 618, 1705, 928, 1053, 433, 631, 526, 703, 391, 486, 563, 446, 496, 244, 285, 109, 225, 232, 202, 803, 867, 1063, 467, 861, 1524, 836, 419, 407, 1019, 853, 1532, 493, 718, 975, 83, 318, 1082, 1087, 360, 965, 442, 448, 783, 656, 469, 873, 688, 1517, 196, 1050, 502, 507, 567, 438, 1560, 420, 1572, 658, 533, 651, 1033, 660, 623, 906, 622, 550, 889, 204, 1079, 1460, 1090, 235, 254, 249, 116, 281, 84, 903, 675, 472, 747, 1635, 558, 1059, 1530, 627, 452, 561, 1

In [8]:
train_eval_PaperRank('CiteSeer_umd_5_fold')

CiteSeer_umd_5_fold/train_fold1.txt CiteSeer_umd_5_fold/test_fold1.txt
[1384, 1050, 1026, 127, 112, 735, 939, 1002, 726, 662, 1154, 1393, 926, 1060, 906, 634, 499, 1339, 983, 1826, 303, 521, 1401, 448, 984, 1511, 173, 893, 2035, 1599, 835, 817, 371, 1825, 957, 1914, 2430, 2082, 1507, 2055, 675, 1408, 691, 1776, 318, 1112, 993, 921, 450, 1150, 586, 923, 249, 1006, 420, 793, 1447, 1718, 812, 2474, 228, 375, 925, 732, 263, 224, 644, 555, 290, 451, 575, 834, 1036, 1392, 314, 1509, 440, 807, 1090, 563, 808, 687, 943, 1787, 270, 520, 728, 452, 597, 681, 873, 678, 1075, 1049, 599, 1152, 892, 656, 337, 128, 1108, 824, 202, 1153, 798, 317, 1088, 814, 275, 1372, 1369, 5, 839, 932, 36, 426, 295, 2119, 460, 16, 1666, 702, 508, 1312, 654, 172, 653, 1125, 1534, 811, 1003, 265, 688, 1091, 998, 439, 1663, 1470, 2088, 1157, 745, 1573, 470, 730, 1014, 867, 729, 260, 776, 481, 959, 670, 685, 474, 1185, 773, 238, 995, 1916, 1203, 145, 697, 671, 842, 471, 765, 410, 264, 663, 1821, 1113, 310, 974, 226, 568,

In [9]:
train_eval_PaperRank('AlgoCitation_5_fold')

AlgoCitation_5_fold/train_fold1.txt AlgoCitation_5_fold/test_fold1.txt
[3892, 4856, 4915, 5033, 573, 4834, 5026, 5006, 5210, 4895, 930, 1733, 2562, 1433, 4849, 4971, 505, 4855, 4885, 6031, 4916, 3389, 5201, 4978, 4990, 3044, 5452, 5266, 4917, 5344, 5744, 6003, 5616, 4981, 5162, 4691, 4944, 4338, 2015, 841, 4872, 5513, 5595, 3394, 4744, 3033, 1695, 5367, 5107, 4920, 5218, 2580, 1623, 5505, 4108, 4992, 5698, 5335, 2568, 5619, 238, 5062, 4942, 5012, 1162, 803, 2083, 4152, 3608, 1876, 1844, 3364, 3840, 5420, 197, 6058, 4999, 1610, 3971, 2450, 4851, 5447, 5251, 2139, 4973, 3624, 4946, 5641, 6431, 3694, 5346, 5194, 5166, 5623, 1995, 4153, 4046, 3061, 1682, 5974, 1465, 601, 5393, 4886, 5415, 4275, 551, 5373, 5511, 4969, 4194, 5009, 3503, 5560, 5182, 4869, 2702, 1022, 5105, 4976, 5113, 5613, 6418, 3101, 5130, 2602, 1993, 1840, 2404, 3593, 1653, 3014, 4589, 5496, 5225, 7607, 6355, 5417, 1411, 1602, 2036, 1956, 2810, 4714, 1313, 5016, 468, 1975, 8150, 3609, 5384, 5554, 3790, 334, 1469, 6718, 623

In [26]:
# !tar cvfz PaperRank_score.tar.gz PaperRank_score

PaperRank_score/
PaperRank_score/CiteULike_5_fold_fold3.pkl
PaperRank_score/CiteULike_5_fold_fold5.pkl
PaperRank_score/.ipynb_checkpoints/
PaperRank_score/CiteSeer_umd_5_fold_fold3.pkl
PaperRank_score/AlgoCitation_5_fold_fold5.pkl
PaperRank_score/CiteSeer_umd_5_fold_fold2.pkl
PaperRank_score/TopicCitation_5_fold_fold5.pkl
PaperRank_score/TopicCitation_5_fold_fold1.pkl
PaperRank_score/AlgoCitation_5_fold_fold1.pkl
PaperRank_score/TopicCitation_5_fold_fold4.pkl
PaperRank_score/CiteULike_5_fold_fold4.pkl
PaperRank_score/AlgoCitation_5_fold_fold4.pkl
PaperRank_score/CiteSeer_umd_5_fold_fold5.pkl
PaperRank_score/CiteULike_5_fold_fold1.pkl
PaperRank_score/CiteSeer_umd_5_fold_fold1.pkl
PaperRank_score/CiteULike_5_fold_fold2.pkl
PaperRank_score/AlgoCitation_5_fold_fold3.pkl
PaperRank_score/TopicCitation_5_fold_fold2.pkl
PaperRank_score/AlgoCitation_5_fold_fold2.pkl
PaperRank_score/TopicCitation_5_fold_fold3.pkl
PaperRank_score/CiteSeer_umd_5_fold_fold4.pkl


///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////